In [62]:
from lxml import html
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [26]:
def get_page(page_number):
    page = requests.get('https://www.allabolag.se/where/stockholm/page/' + str(page_number))
    soup = BeautifulSoup(page.content, 'html.parser')
    res = soup.find_all('h2', class_="search-results__item__title") 
    list_url = [x.find('a').get('href') for x in res]
    return list_url

In [27]:
get_page(1)

[u'https://www.allabolag.se/1144015110/u-o-bergstedts-fond',
 u'https://www.allabolag.se/1144015128/mathilde-brandts-minnesstiftelse',
 u'https://www.allabolag.se/1144015136/peter-brandts-gavofond',
 u'https://www.allabolag.se/1144015144/peter-brandts-pensionsfond',
 u'https://www.allabolag.se/1144015151/peter-brandts-understodsfond',
 u'https://www.allabolag.se/1144015169/peter-brandts-uppfostringsfond',
 u'https://www.allabolag.se/1144015201/nauclerska-stiftelsen-grisslehamn',
 u'https://www.allabolag.se/1144015219/clara-rosengrens-pensionsfond',
 u'https://www.allabolag.se/1140011097/bergstedts-fond-j-a-jens-bergstedt',
 u'https://www.allabolag.se/1140011105/bergstedts-fond-t-l',
 u'https://www.allabolag.se/1136009154/silfverstolpeska-slaktforeningen',
 u'https://www.allabolag.se/1136009162/stift-erik-ehlins-understodsfonder',
 u'https://www.allabolag.se/1136010053/stift-erik-ehlins-u-fond-f-s-ehlins-fam',
 u'https://www.allabolag.se/5567350086/sandsjo-soner-systemutveckling-ab',
 u

In [144]:
def get_company_info(url):
    
    c_dict = {}
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    company_table = soup.find('table', class_="company-table") 
    company_name = url.split('/')[-1].replace('-', ' ') 
    
    if company_table:
        data_rows = company_table.find_all('tr')
        years = company_table.find_all('th', class_='data-pager__page')
        
        for row in data_rows:
            
            if row.th and row.th.text:
                name = row.th.text.strip()
                if name == 'Bokslut & nyckeltal':
                    continue
                values = row.find_all('td')
                
                data=[]
                company_names=[]
                for value in values:    
                    clean_value = value.text.strip().replace('%', '').replace(',','.')
                    if clean_value:
                        try:
                            data.append(float(clean_value))
                            company_names.append(company_name)
                        except ValueError:
                            data.append(0)
                        
                c_dict[name] = data
                c_dict['company_name'] = company_names
        
        years_list = []        
        for row in years:
            if row.text:
                year = row.text.strip().split('-')[0]
                years_list.append(year)
        c_dict['year'] = years_list
        
        
        df = pd.DataFrame(data = c_dict).set_index('company_name')
        
    return df         

In [40]:
company='https://www.allabolag.se/5567350086/sandsjo-soner-systemutveckling-ab'

In [145]:
get_company_info(company)

,Antal anställda,Kassalikviditet,Löner till styrelse & VD (tkr),Löner till övriga anställda (tkr),Omsättning (tkr),Res. e. finansnetto (tkr),Soliditet,Summa tillgångar (tkr),Vinstmarginal,year,Årets resultat (tkr)
company_name,,,,,,,,,,,
sandsjo soner systemutveckling ab,1.0,406.35,0,0.0,0,597.0,83.33,0,35.22,2017,422.0
sandsjo soner systemutveckling ab,1.0,263.90,0,416.0,0,661.0,79.51,0,36.42,2016,365.0
sandsjo soner systemutveckling ab,1.0,214.36,0,432.0,0,915.0,80.30,0,51.20,2015,552.0
sandsjo soner systemutveckling ab,1.0,270.68,0,394.0,0,721.0,76.67,0,42.29,2014,452.0
sandsjo soner systemutveckling ab,1.0,263.60,0,396.0,0,380.0,63.95,0,22.53,2013,190.0


In [139]:
def scrape_pages(n):
    n = 10
    for i in range(1,n+1):
        urls = get_page(i)

        for url in (urls):
            try:
                print (get_company_info(url))
            except:
                pass


   Antal anställda  Kassalikviditet  Löner till styrelse & VD (tkr)  \
0              1.0           406.35                               0   
1              1.0           263.90                               0   
2              1.0           214.36                               0   
3              1.0           270.68                               0   
4              1.0           263.60                               0   

   Löner till övriga anställda (tkr)  Omsättning (tkr)  \
0                                0.0                 0   
1                              416.0                 0   
2                              432.0                 0   
3                              394.0                 0   
4                              396.0                 0   

   Res. e. finansnetto (tkr)  Soliditet  Summa tillgångar (tkr)  \
0                      597.0      83.33                       0   
1                      661.0      79.51                       0   
2                      